#### 사용자 정의 모델 클래스
 - 부모 클래스 : nn.Module
 - 필수오버라이딩 : 
  * __ init __() : 모델 층 구성 => 설계
  * foward() : 순방향 학습 진행 코드 구현

In [15]:
## 모듈로딩
import torch                                # 텐서 관련 모듈
import torch.nn as nn                       # 인공신경망 관련 모듈
import torch.nn.functional as F             # 인공신경망 관련 함수들 모듈(손실함수, 활성화함수 등)
import torch.optim as optim                 # 최적화 관련 모듈(가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary               #모델 구조 및 정보 관련 모듈
from torchmetrics.regression import *       # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *   # 분류 성능 지표 관련 모듈



In [16]:
# 랜덤 고정
torch.manual_seed(10)

# 텐서 저장 및 실행 위치
DEVICE = 'cuda'if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

- [기본] 신경망클래스 <hr>
    * 입력층 - 입력 피쳐 고정
    * 출력층 - 출력 타겟 고정
    * 은닉층 - 고정

In [17]:
# 모델 설계
# 데이터셋 : 피쳐 4개, 타겟 1개, 회귀
# 입력층 : 입력     4개     출력  20개     AF   ReLU
# 은닉층 : 입력    20개     출력  100개    AF   ReLU
# 출력층 : 입력   100개     출력  1개      AF   Sigmoid & Softmax(분류일 경우 확률로 바꿔줘야 하니 sigmoid 이진분류에는 sigmoid, 다중분류에는(Softmax) 사용)

In [18]:
# 모든 입력이 고정적인 모델
class MyModel(nn.Module) :
    
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self) :
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(4,20)          # w 4 + b 1 => 1p, 5 * 20 = 100개 변수
        self.hidden_layer = nn.Linear(20,100)       # w 20 + b 1 => 1p, 21 * 100 = 2100개 변수
        self.out_layer = nn.Linear(100,1)           # w 100 + b 1 => 1p, 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자(x) : 학습용 데이터
    def forward(self, x) :
        print('calling foward()')
        y = self.input_layer(x)
        y = F.relu(y)
        
        y = self.hidden_layer(y)
        y = F.relu(y)

        self.out_layer(y)

        return self.out_layer(y)

In [19]:
# 입력 피쳐 수가 동적인 모델
class MyModel2(nn.Module) :
    
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_features) :
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_features,20)          # w 4 + b 1 => 1p, 5 * 20 = 100개 변수
        self.hidden_layer = nn.Linear(20,100)       # w 20 + b 1 => 1p, 21 * 100 = 2100개 변수
        self.out_layer = nn.Linear(100,1)           # w 100 + b 1 => 1p, 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자(x) : 학습용 데이터
    def forward(self, x) :
        print('calling foward()')
        y = self.input_layer(x)
        y = F.relu(y)
        
        y = self.hidden_layer(y)
        y = F.relu(y)

        self.out_layer(y)

        return self.out_layer(y)

In [20]:
# 입력 피쳐 수와 은닉층 퍼셉트론 수가 동적인 모델
class MyModel3(nn.Module) :
    
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_features, in_out, h_out) :
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_features,in_out)          # w 4 + b 1 => 1p, 5 * 20 = 100개 변수
        self.hidden_layer = nn.Linear(in_out,h_out)       # w 20 + b 1 => 1p, 21 * 100 = 2100개 변수
        self.out_layer = nn.Linear(h_out,1)           # w 100 + b 1 => 1p, 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자(x) : 학습용 데이터
    def forward(self, x) :
        print('calling foward()')
        y = self.input_layer(x)
        y = F.relu(y)
        
        y = self.hidden_layer(y)
        y = F.relu(y)

        self.out_layer(y)

        return self.out_layer(y)

In [21]:
# 은닉층의 개수가 동적인 모델
class MyModel4(nn.Module) :
    
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_features, in_out, h_out) :
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_features, in_out)          # w 4 + b 1 => 1p, 5 * 20 = 100개 변수
        for i in range(5) :
            self.hidden_layer = nn.Linear(in_out, h_out) 

 
        self.out_layer = nn.Linear(h_out,1)           # w 100 + b 1 => 1p, 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자(x) : 학습용 데이터
    def forward(self, x) :
        print('calling foward()')
        y = self.input_layer(x)
        y = F.relu(y)
        
        y = self.hidden_layer(y)
        y = F.relu(y)

        self.out_layer(y)

        return self.out_layer(y)

In [22]:
## 모델 인스턴스 생성
m1 = MyModel()

In [23]:
## 모델 파라미터 => w와 b 확인
for m in m1.named_parameters() :print(m)

('input_layer.weight', Parameter containing:
tensor([[-0.0419, -0.0171, -0.1875,  0.1150],
        [-0.2861, -0.0882,  0.1938,  0.4693],
        [ 0.1178, -0.1696,  0.0479, -0.0560],
        [ 0.2041,  0.0573,  0.1959,  0.4849],
        [-0.2076, -0.0177,  0.1150, -0.0033],
        [-0.0479, -0.4425, -0.4313, -0.4499],
        [-0.4892, -0.4657, -0.3788, -0.4510],
        [-0.4690,  0.2192,  0.3067,  0.3379],
        [ 0.2694,  0.1694,  0.2203, -0.2765],
        [ 0.4502, -0.0345,  0.4314,  0.1533],
        [ 0.3914,  0.3988, -0.1045, -0.1454],
        [ 0.0752, -0.0213,  0.0782,  0.2536],
        [-0.3907, -0.0229, -0.3924,  0.4829],
        [-0.3517,  0.0956, -0.1366,  0.2842],
        [ 0.0017, -0.0503,  0.3660,  0.4567],
        [-0.3629, -0.4823,  0.0417,  0.1575],
        [ 0.1141,  0.4619,  0.2244, -0.2300],
        [-0.3424,  0.3879,  0.4792, -0.2373],
        [-0.3200,  0.1750, -0.3576, -0.1210],
        [-0.4945,  0.1368, -0.1705, -0.2797]], requires_grad=True))
('input_layer

In [24]:
# 학습 진행 ==> 모델 인스턴스명(데이터)
# 임의의 데이터 생성
dataTS = torch.FloatTensor([[1,3,5,7],[2,4,6,8]])
targetTS = torch.FloatTensor([[4],[5]])

# 학습
pre_y = m1(dataTS)
print(pre_y)

calling foward()
tensor([[-0.4545],
        [-0.5435]], grad_fn=<AddmmBackward0>)
